In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from sklearn.metrics import f1_score
import torch
import torchvision
import torchvision.transforms as transforms
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.utils import _pair, _quadruple
from scipy import stats
import numpy as np

In [10]:
class PixelBinning(nn.Module):
    def __init__(self):
        super(PixelBinning, self).__init__()
    def _padding(self, x):
        ph = 0
        pw = 0
        ih, iw = x.size()[2:] # getting image height and width
        if ih % 4 != 0:
            ph = 4 - ih % 4
        if iw % 4 != 0:
            pw = 4 - iw % 4  
        pl = pw // 2
        pr = pw - pl
        pt = ph // 2
        pb = ph - pt
        padding = (pl, pr, pt, pb)
        return padding
    def forward(self, x):
        # we assume x has same height and width
        output_height = x.shape[2]//2
        output_weight = x.shape[3]//2
        x = F.pad(x, self._padding(x), mode='reflect')
        unfold = nn.Unfold(kernel_size=(2, 2), dilation=2)
        x_shape = x.shape
        x = unfold(x)
        x = x.reshape(x_shape[0], x_shape[1], 4, -1)
        temp = []
        for i in range(x.size()[3]):
          lst = []
          for j in range(x.size()[2]):
            lst.append(x[0][0][j][i])
          temp.append(lst)
        temp = torch.Tensor(temp)
        temp = temp.reshape(6, 6, -1)
        temp = temp[:,[0,1,4,5],:]
        temp = temp[[0,1,4,5], :, :]
        print(temp.size())
        print("temp", temp)
        x = x.sum(dim=2)/4.0
        x = x.reshape(x_shape[0], x_shape[1], x_shape[2]//2 + (x_shape[2]//4 - 1)*2, -1)
        idx = torch.zeros(x_shape[2]//2, dtype=torch.long)
        #inefficient operation start
        cnt = 0
        i = 0
        for k in range(x_shape[2]//2):
          idx[k] = i
          i += 1
          cnt += 1
          if cnt == 2:
            cnt = 0
            i += 2
        #inefficient operation end
        x = x[:, :, idx, :]
        x = x[:, :, :, idx]
        return x[:,:, :output_height, :output_weight]

In [11]:
input = torch.arange(49, dtype=torch.float).reshape(1,1,7,7)
print('input')
print(input)
print('input size')
print(input.size())
pixelbinning = PixelBinning()
output = pixelbinning(input)
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
print('output size')
print(output.size())
output = maxpool(input)
print('output size')
print(output.size())

input
tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.],
          [ 7.,  8.,  9., 10., 11., 12., 13.],
          [14., 15., 16., 17., 18., 19., 20.],
          [21., 22., 23., 24., 25., 26., 27.],
          [28., 29., 30., 31., 32., 33., 34.],
          [35., 36., 37., 38., 39., 40., 41.],
          [42., 43., 44., 45., 46., 47., 48.]]]])
input size
torch.Size([1, 1, 7, 7])
torch.Size([4, 4, 4])
temp tensor([[[ 0.,  2., 14., 16.],
         [ 1.,  3., 15., 17.],
         [ 4.,  6., 18., 20.],
         [ 5.,  5., 19., 19.]],

        [[ 7.,  9., 21., 23.],
         [ 8., 10., 22., 24.],
         [11., 13., 25., 27.],
         [12., 12., 26., 26.]],

        [[28., 30., 42., 44.],
         [29., 31., 43., 45.],
         [32., 34., 46., 48.],
         [33., 33., 47., 47.]],

        [[35., 37., 35., 37.],
         [36., 38., 36., 38.],
         [39., 41., 39., 41.],
         [40., 40., 40., 40.]]])
output size
torch.Size([1, 1, 3, 3])
output size
torch.Size([1, 1, 3, 3])
